In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
N = 1
M = 2
E = 2
envModel = Env('MyEnv', E)
envName = ['Stage1', 'Stage2']
envType = ['UP', 'DOWN']

rate = np.array([[2,1],[1,2]])

In [3]:
def example_randomEnvironment_genqn(rate, N):
    # sn1
    qnet = Network('qn1')
    
    node = np.empty(2, dtype=object)
    node[0] = Delay(qnet, 'Queue1')
    node[1] = Queue(qnet, 'Queue2', SchedStrategy.PS)

    jobclass = np.empty(1, dtype=object)
    jobclass[0] = ClosedClass(qnet, 'Class1', N, node[0], 0)
    
    node[0].setService(jobclass[0], Exp(rate[0]))
    node[1].setService(jobclass[0], Exp(rate[1]))
    
    P = qnet.initRoutingMatrix()
    P.set(jobclass[0],jobclass[0], [[0,1],[1,0]])
    qnet.link(P)
    return qnet

In [4]:
envSubModel = [example_randomEnvironment_genqn(rate[:,0],N), example_randomEnvironment_genqn(rate[:,1],N)]

for e in range(E):
    envModel.addStage(e, envName[e], envType[e], envSubModel[e])
 
envRates = [[0,1], [0.5,0.5]]
for e in range(E):
    for h in range(E):
        if envRates[e][h]>0.0:
            envModel.addTransition(e, h, Exp(envRates[e][h]))


The metasolver considers an environment with 2 stages and a queueing network with 2 stations.
Every time the stage changes, the queueing network will modify the service rates of the stations.

In [5]:

##options.iter_tol = 1e-5
# options = Solver.defaultOptions
# options.timespan = [0,Inf]
# options.iter_max = 100
# options.iter_tol = 0.01
# options.method = 'default'
# options.verbose = true

#TODO: getStageTable not implemented yet
#envModel.getStageTable() 

#TODO: refactor Java Env to be closer to MATLAB 
# sfoptions = SolverFluid.defaultOptions
# sfoptions.timespan = [0,1e3]
# sfoptions.verbose = false
 
solvers = np.empty(2, dtype=object)
solvers[0] = SolverFluid(envSubModel[0])
solvers[1] = SolverFluid(envSubModel[1])
# envSolver = SolverEnv(envModel,@(model) SolverFluid(model, sfoptions),options)
solver = SolverEnv(envModel, solvers)
# [QN,UN,TN] = envSolver.getAvg()
avgTable = solver.getAvgTable()
print(avgTable)



End time of transient analysis unspecified, setting the timespan option to [0.000000, 30.000000].
End time of transient analysis unspecified, setting the timespan option to [0.000000, 30.000000].
  Station JobClass    QLen    Util     RespT  ResidT  ArvR      Tput
0  Queue1   Class1  0.5584  0.5584  0.805633     0.0   0.0  0.693119
1  Queue2   Class1  0.4416  0.4416  0.645061     0.0   0.0  0.684587
